In [18]:
import json 
import os
from collections import Counter

with open("Ground_Truth.json", "r") as file:
    ground_truth = json.load(file)
path = 'QwenOCR'
file_paths = f'/home/jovyan/Evaluation/OCR/Data/{path}.json'
with open(file_paths, "r") as file:
    labels_list = json.load(file)
    
data = {}
total_recall = 0
total_precision = 0
invalid = 0 
# Goes through each image
for file in labels_list:
    # Counts the number of times a word repeats 
    predicted_counter = Counter(labels_list[file])
    truth_counter = Counter(ground_truth[file])

    correct_words = []
    wrong_words = []

    correct = 0
    wrong = 0
    # Goes through each label
    for word, pred_count in predicted_counter.items():
        # Finds how many of that word is repeated in the ground truth
        true_count = truth_counter[word]
        # Compares the how many repeats of the word in the output to the ground truth
        matched_count = min(pred_count, true_count)
        # If it matches, counts it as correct 
        correct += matched_count
        wrong += pred_count - matched_count
        
        correct_words.extend([word] * matched_count)
        wrong_words.extend([word] * (pred_count - matched_count))

    if len(ground_truth[file]) != 0:
        recall = correct / len(ground_truth[file])
        precision = correct / (correct + wrong)
        total_recall += recall
        total_precision += precision
        data[file] = {
            "Correct Words": correct_words,
            "Incorrect Words": wrong_words,
            "Ground Truth": ground_truth[file],
            "Recall": round(recall * 100, 2),
            "Precision": round(precision * 100, 2)
        }
    else:
        data[file] = {
            "Correct Words": 0,
            "Incorrect Words": 0,
            "Ground Truth": 0,
            "Recall": 0,
            "Precision": 0
        }
        invalid += 1

overall_recall = total_recall/(len(labels_list) - invalid)
overall_precision = total_precision/(len(labels_list) - invalid)
with open(f"{path}(Labels).json", "w") as f:
    json.dump(data, f, indent=4)
print(f"File Created Successfully")       
print(f"Recall: {round(overall_recall*100,2)}")
print(f"Precision: {round(overall_precision*100,2)}")

File Created Successfully
Recall: 73.39
Precision: 76.36
